In [30]:
# !pip3 install wikipedia-api huggingface_hub lancedb pylance pydantic tqdm

In [ ]:
# setup a HF API Key at https://huggingface.co/settings/tokens
HF_API_KEY = "XXXXXXXXXX"

In [130]:
import wikipediaapi
import requests
import lancedb
from tqdm import tqdm
from huggingface_hub import InferenceClient
from lancedb.pydantic import LanceModel, Vector

# scrape wikipedia
def get_wiki_article(title):
    # wikipedia wants this for whatever reason
    user_agent = 'YourAppName/1.0 (YourContactInfo)'
    wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
    return wiki_wiki.page(title).text

# chunk a string into max chunk_size words
def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = words[i:i + chunk_size]
        chunks.append(' '.join(chunk))

    return chunks

# embed a list of strings
def embed(text_items, batch_size=8):
    # always pass a list of strings
    text_items = [text_items] if isinstance(text_items, str) else text_items
    
    hf_client = InferenceClient(api_key=HF_API_KEY)

    # Process text items in batches
    embeddings = []
    for i in tqdm(
        range(0, len(text_items), batch_size),
        "Embedding.."
    ):
        batch = text_items[i:i + batch_size]
        batch_embeddings = hf_client.feature_extraction(
            text=batch, model="mixedbread-ai/mxbai-embed-large-v1"
        )
        embeddings.extend(batch_embeddings)

    return embeddings

# search vector store for query (str)
def retrieve(query):
    query = embed(query)
    
    return table.search(query).limit(5).to_pydantic(TextChunk)

question = "How many Turkish people live in Paris?"

# ask LLM providing chunks as context
def generate(question, retrieved_chunks):
    msg_template = """Please answer the question using only the provided Wikipedia excerpts.
    
    Question: {question}
    
    Relevant Wikipedia excerpts:
    {chunks}"""
    
    msg = msg_template.format(
        question=question,
        chunks="\n\n".join([f"Excerpt #{i+1}: {result.text}" for i, result in enumerate(retrieved_chunks)])
    )
    
    hf_client = InferenceClient(api_key=HF_API_KEY)    
    output = hf_client.chat_completion(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[
            dict(role="system", content="You are a helpful assistant."),
            dict(role="user", content=msg),
        ],
        max_tokens=2048,
    )
    
    return " ".join([choice.message.content for choice in output.choices])

class TextChunk(LanceModel):
    # emb. dimension of mxbai-embed-large-v1 = 1024
    vector: Vector(1024)
    text: str 
    doc_title: str


In [125]:
wiki_article = "Paris"

# fetch wikipedia article, chunk and embed
text = get_wiki_article(wiki_article)
text_chunks = chunk_text(text)
text_embeddings = embed(text_chunks)

# ready chunks for DB
lance_chunks = [
    dict(vector=emb, text=text, doc_title=wiki_article)
    for emb, text in zip(text_embeddings, text_chunks)
]

# store in vector DB
db = lancedb.connect("vectorstore") # can be any name, will create a directory where vectors are stored
table = db.create_table("wiki_docs", schema=TextChunk, exist_ok=True)
table.add(lance_chunks)

Embedding..: 100%|████████████████████████████████| 4/4 [00:11<00:00,  2.95s/it]


AddResult(version=2)

In [133]:
# retrieve and generate 
question = "where are most immigrants in Paris from?"

retrieved_chunks = retrieve(question)
answer = generate(question, retrieved_chunks)

print(answer)

Embedding..: 100%|████████████████████████████████| 1/1 [00:00<00:00,  3.29it/s]


To find where most immigrants in Paris are from, we need to analyze the provided Wikipedia excerpts. Excerpt #1 mentions the immigrant population in the City of Paris and the Paris Region in 2012:

- In the City of Paris: 
  - Europe: 135,853
  - Maghreb: 112,369
  - Sub-Saharan Africa and Egypt: 70,852
  - Turkey: 5,059
  - Asia outside Turkey: 91,297
  - Americas: 38,858
  - South Pacific: 1,365

- In the Paris Region:
  - Europe: 590,504
  - Maghreb: 627,078
  - Sub-Saharan Africa and Egypt: 435,339
  - Turkey: 69,338
  - Asia outside Turkey: 322,330
  - Americas: 113,363
  - South Pacific: 2,261

From the above data, it's clear that the majority of immigrants in Paris are from Europe and the Maghreb region.
